In [ ]:
from polars.testing import assert_frame_equal

In [ ]:
#using a seperate connection so test tables dont live in the production connection
test_con = duckdb.connect(database = ':memory:', read_only=False)

In [ ]:
def happy_flow_clean_rentals_for_silver(connection):

    logger.info('Started unit test: happy_flow_clean_rentals_for_silver')

    rents_test_query ="""
    CREATE TABLE IF NOT EXISTS rents_test (
        rent VARCHAR,
        postalCode VARCHAR
    );

    INSERT INTO rents_test (rent, postalCode) VALUES
    ('€ 1,-', '1234AB'),
    ('€ 999,-', '5678CD'),
    ('€ 800,- Utilities incl.', '2345EF'),
    ('€ 1234,- Extra fee', '3456GH'),
    ('€ 50,- incl.', 'ABCD12'),  
    ('Invalid entry', '123ABCD');
    """

    rents_test_expected_query="""
    CREATE TABLE rents_test_expected (
    zipcode VARCHAR,
    price INT
    );

    INSERT INTO rents_test_expected (zipcode, price) VALUES
    ('1234 AB', 1),
    ('5678 CD', 999),
    ('2345 EF', 800),
    ('3456 GH', 1234);
    """

    logger.info('creating test and expected data tables')
    connection.sql(rents_test_query)
    connection.sql(rents_test_expected_query)

    #no need to log this as the function has logging itself
    clean_rentals_for_silver(connection, table='rents_test')

    logger.info('setting up cleaned and expected data table in order for comparison')

    #ordering and converting to polars dataframe for easy comparison
    cleaned_test_set = connection.sql('FROM rentals_cleaned ORDER BY price desc').pl()
    expected_test_set = connection.sql('FROM rents_test_expected ORDER BY price desc').pl()

    logger.info('Checking cleaned test data against expected cleaned test data')
    #DEBUG
    #logger.info(f'Cleaned test set: \n {cleaned_test_set}')
    #logger.info(f'Expected test set: \n {expected_test_set}')

    assert_frame_equal(cleaned_test_set, expected_test_set)

    logger.info('Successfully completed unit test: happy_flow_clean_rentals_for_silver')


    

In [ ]:
#start unit test
happy_flow_clean_rentals_for_silver(test_con)


In [ ]:
test_con.close()